# Microsoft Agent Frameworkを使ったHuman-in-the-Loopワークフロー

## 🎯 学習目標

このノートブックでは、Microsoft Agent Frameworkの`RequestInfoExecutor`を使用して**Human-in-the-Loop**ワークフローを実装する方法を学びます。この強力なパターンを使うことで、AIワークフローを一時停止して人間の入力を収集し、エージェントをインタラクティブにし、重要な決定に対する人間のコントロールを可能にします。

## 🔄 Human-in-the-Loopとは？

**Human-in-the-loop (HITL)**は、AIエージェントが実行を一時停止し、人間の入力を要求してから続行するデザインパターンです。これは以下の目的で重要です：

- ✅ **重要な決定** - 重要な行動を取る前に人間の承認を得る
- ✅ **曖昧な状況** - AIが不確実な場合に人間が明確化する
- ✅ **ユーザーの好み** - 複数の選択肢からユーザーに選んでもらう
- ✅ **コンプライアンスと安全性** - 規制された操作に対する人間の監視を確保する
- ✅ **インタラクティブな体験** - ユーザー入力に応答する会話型エージェントを構築する

## 🏗️ Microsoft Agent Frameworkでの仕組み

フレームワークはHITLのために以下の3つの主要コンポーネントを提供します：

1. **`RequestInfoExecutor`** - ワークフローを一時停止し、`RequestInfoEvent`を発行する特別なエグゼキューター
2. **`RequestInfoMessage`** - 人間に送信される型付きリクエストペイロードの基底クラス
3. **`RequestResponse`** - `request_id`を使用して人間の応答を元のリクエストと関連付ける

**ワークフローパターン:**
```
Agent detects need for input
    ↓
Sends message to RequestInfoExecutor
    ↓
Workflow pauses & emits RequestInfoEvent
    ↓
Application collects human input (console, UI, etc.)
    ↓
Application sends RequestResponse via send_responses_streaming()
    ↓
Workflow resumes with human input
```

## 🏨 例: ユーザー確認付きホテル予約

条件付きワークフローを基にして、代替目的地を提案する**前に**人間の確認を追加します：

1. ユーザーが目的地をリクエスト（例:「パリ」）
2. `availability_agent`が部屋の空き状況を確認
3. **部屋がない場合** → `confirmation_agent`が「代替案を見たいですか？」と尋ねる
4. ワークフローが`RequestInfoExecutor`を使用して**一時停止**
5. **人間がコンソール入力で**「はい」または「いいえ」と応答
6. `decision_manager`が応答に基づいてルートを決定：
   - **はい** → 代替目的地を表示
   - **いいえ** → 予約リクエストをキャンセル
7. 最終結果を表示

これにより、エージェントの提案に対するユーザーのコントロールを実現します！

---

さあ、始めましょう！ 🚀


## ステップ 1: 必要なライブラリをインポートする

標準のエージェントフレームワークコンポーネントに加えて、**人間が介入する特定のクラス**をインポートします:
- `RequestInfoExecutor` - 人間の入力を待つためにワークフローを一時停止するエグゼキューター
- `RequestInfoEvent` - 人間の入力が要求された際に発生するイベント
- `RequestInfoMessage` - 型付きリクエストペイロードの基底クラス
- `RequestResponse` - 人間の応答とリクエストを関連付ける
- `WorkflowOutputEvent` - ワークフローの出力を検出するためのイベント


In [21]:
import asyncio
import json
import os
from dataclasses import dataclass
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Executor,
    RequestInfoEvent,          # NEW: Event when human input is requested
    RequestInfoExecutor,       # NEW: Executor that gathers human input
    RequestInfoMessage,        # NEW: Base class for request payloads
    RequestResponse,           # NEW: Correlates response with request
    Role,
    WorkflowBuilder,
    WorkflowContext,
    WorkflowOutputEvent,       # NEW: Event for workflow outputs
    WorkflowRunState,          # NEW: Enum of workflow run states
    WorkflowStatusEvent,       # NEW: Event for run state changes
    ai_function,
    executor,
    handler,                   # NEW: Decorator for executor methods
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")
print("🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse")

✅ All imports successful!
🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse


## ステップ2: 構造化された出力のためのPydanticモデルを定義する

これらのモデルは、エージェントが返す**スキーマ**を定義します。条件付きワークフローのすべてのモデルを保持し、以下を追加します:

**Human-in-the-Loop向けの新機能:**
- `HumanFeedbackRequest` - 人間に送信されるリクエストペイロードを定義する`RequestInfoMessage`のサブクラス  
  - `prompt`（質問内容）と`destination`（利用できない都市に関するコンテキスト）を含む


In [22]:
# Existing models from conditional workflow
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""
    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""
    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""
    destination: str
    action: str
    message: str


# NEW: Pydantic model for agent's response format
class ConfirmationQuestion(BaseModel):
    """
    Pydantic model used by confirmation_agent's response_format.
    This is what the agent will output as JSON.
    """
    question: str  # The question to ask the user
    destination: str  # The unavailable destination for context


# NEW: Dataclass for RequestInfoExecutor
@dataclass
class HumanFeedbackRequest(RequestInfoMessage):
    """
    Request sent to RequestInfoExecutor asking if user wants alternatives.
    
    MUST be a dataclass subclassing RequestInfoMessage for type compatibility.
    This is what gets sent to the RequestInfoExecutor.
    """
    prompt: str = ""  # The question to ask the user
    destination: str = ""  # The unavailable destination for context


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")
print("   - ConfirmationQuestion (agent response format) 🆕")
print("   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)
   - ConfirmationQuestion (agent response format) 🆕
   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕


## ステップ3: ホテル予約ツールを作成する

条件付きワークフローと同じツール - 目的地で部屋が利用可能かどうかを確認します。


In [23]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## ステップ4: ルーティングの条件関数を定義する

人間が介入するワークフローのために、**4つの条件関数**が必要です:

**条件付きワークフローから:**
1. `has_availability_condition` - ホテルが利用可能な場合にルートする
2. `no_availability_condition` - ホテルが利用不可の場合にルートする

**人間が介入する新しい条件:**
3. `user_wants_alternatives_condition` - ユーザーが代替案に「はい」と答えた場合にルートする
4. `user_declines_alternatives_condition` - ユーザーが代替案に「いいえ」と答えた場合にルートする


In [24]:
# Existing condition functions from conditional workflow
def has_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels ARE available."""
    if not isinstance(message, AgentExecutorResponse):
        return True

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )
        return result.has_availability
    except Exception as e:
        display(HTML(f"""<div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'><strong>⚠️  Error:</strong> {str(e)}</div>"""))
        return False


def no_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels are NOT available."""
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )
        return not result.has_availability
    except Exception as e:
        return False


# NEW: Condition functions for human-in-the-loop routing
def user_wants_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user WANTS to see alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            wants_alternatives = "wants to see alternative" in msg_text or "want to see alternative" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
                    <strong>🔍 User Decision:</strong> User wants alternatives = <strong>{wants_alternatives}</strong>
                </div>
            """)
            )
            
            return wants_alternatives
    
    return False
def user_declines_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user DECLINES alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            declined = "declined" in msg_text or "has declined" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fce4ec; border-left: 4px solid #c2185b; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 User Decision:</strong> User declined alternatives = <strong>{declined}</strong>
                </div>
            """)
            )
            
            return declined
    
    return False
print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")
print("   - user_wants_alternatives_condition (routes when user says yes) 🆕")
print("   - user_declines_alternatives_condition (routes when user says no) 🆕")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)
   - user_wants_alternatives_condition (routes when user says yes) 🆕
   - user_declines_alternatives_condition (routes when user says no) 🆕


## ステップ5: Decision Manager Executorを作成する

これが**人間を介したループパターンの核**です！`DecisionManager`はカスタムの`Executor`であり、以下を行います：

1. **人間のフィードバックを受け取る** `RequestResponse`オブジェクトを介して
2. **ユーザーの決定を処理する**（はい/いいえ）
3. **ワークフローをルーティングする** 適切なエージェントにメッセージを送信することで

主な特徴：
- `@handler`デコレーターを使用してメソッドをワークフローステップとして公開
- 元のリクエストとユーザーの回答の両方を含む`RequestResponse[HumanFeedbackRequest, str]`を受け取る
- 条件関数をトリガーするシンプルな「はい」または「いいえ」のメッセージを生成


In [25]:
class DecisionManager(Executor):
    """
    Coordinates workflow routing based on human feedback.
    
    This executor receives RequestResponse objects from the RequestInfoExecutor
    and makes routing decisions by sending simple messages that trigger
    condition functions.
    """

    def __init__(self, id: str | None = None):
        super().__init__(id=id or "decision_manager")

    @handler
    async def on_human_feedback(
        self,
        feedback: RequestResponse[HumanFeedbackRequest, str],
        ctx: WorkflowContext[AgentExecutorRequest],
    ) -> None:
        """
        Process human feedback and let the workflow route based on conditions.
        
        The RequestResponse contains:
        - feedback.data: The user's string reply (e.g., "yes" or "no")
        - feedback.original_request: The HumanFeedbackRequest with context
        
        This handler just displays feedback and passes the RequestResponse through.
        The routing is done by condition functions on the edges.
        """
        user_reply = (feedback.data or "").strip().lower()
        destination = getattr(feedback.original_request, "destination", "unknown")

        display(
            HTML(f"""
            <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
                <strong>🎯 Decision Manager:</strong> Processing user reply: <strong>"{user_reply}"</strong> for {destination}
            </div>
        """)
        )

        if user_reply == "yes":
            display(
                HTML("""
                <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                    <strong>➡️  Routing:</strong> User wants alternatives → Will route to alternative_agent
                </div>
            """)
            )
            # Create and send a message for the alternative_agent
            user_msg = ChatMessage(
                Role.USER,
                text=f"The user wants to see alternative destinations near {destination}. Please suggest one.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        elif user_reply == "no":
            display(
                HTML("""
                <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 Routing:</strong> User declined alternatives → Will route to cancellation_agent
                </div>
            """)
            )
            # Create and send a message for the cancellation_agent
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        else:
            # Handle unexpected input - treat as decline
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                    <strong>⚠️  Warning:</strong> Unexpected input "{user_reply}" - treating as decline
                </div>
            """)
            )
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))


print("✅ DecisionManager executor created with @handler method for human feedback")

✅ DecisionManager executor created with @handler method for human feedback


## ステップ6: カスタム表示エグゼキューターを作成する

条件付きワークフローと同じ表示エグゼキューター - ワークフローの出力として最終結果を生成します。


In [26]:
@executor(id="prepare_human_request")
async def prepare_human_request(
    response: AgentExecutorResponse, 
    ctx: WorkflowContext[HumanFeedbackRequest]
) -> None:
    """
    Transform agent response into HumanFeedbackRequest for RequestInfoExecutor.
    
    This executor bridges the type gap between:
    - confirmation_agent outputs AgentExecutorResponse with ConfirmationQuestion JSON
    - request_info_executor expects HumanFeedbackRequest (RequestInfoMessage dataclass)
    """
    display(
        HTML("""
        <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
            <strong>🔄 Transform:</strong> Converting ConfirmationQuestion to HumanFeedbackRequest
        </div>
    """)
    )
    
    # Parse the agent's Pydantic output (ConfirmationQuestion)
    confirmation = ConfirmationQuestion.model_validate_json(response.agent_run_response.text)
    
    # Convert to HumanFeedbackRequest dataclass for RequestInfoExecutor
    feedback_request = HumanFeedbackRequest(
        prompt=confirmation.question,
        destination=confirmation.destination
    )
    
    # Send the properly typed RequestInfoMessage to the RequestInfoExecutor
    await ctx.send_message(feedback_request)


@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ prepare_human_request executor created with @executor decorator")
print("✅ display_result executor created with @executor decorator")

✅ prepare_human_request executor created with @executor decorator
✅ display_result executor created with @executor decorator


## ステップ7: 環境変数の読み込み

LLMクライアント（GitHub Models、Azure OpenAI、またはOpenAI）を設定します。


In [27]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(
    base_url=os.environ.get("GITHUB_ENDPOINT"), 
    api_key=os.environ.get("GITHUB_TOKEN"), 
    model_id="gpt-4o"
)

print("✅ Chat client configured with GitHub Models")

✅ Chat client configured with GitHub Models


## ステップ8: AIエージェントとエグゼキューターの作成

**6つのワークフローコンポーネント**を作成します:

**エージェント (AgentExecutorでラップ):**
1. **availability_agent** - ツールを使用してホテルの空室状況を確認
2. **confirmation_agent** - 🆕 人間による確認リクエストを準備
3. **alternative_agent** - 代替都市を提案 (ユーザーが「はい」と言った場合)
4. **booking_agent** - 予約を促進 (部屋が空いている場合)
5. **cancellation_agent** - 🆕 キャンセルメッセージを処理 (ユーザーが「いいえ」と言った場合)

**特別なエグゼキューター:**
6. **request_info_executor** - 🆕 `RequestInfoExecutor` 人間の入力のためにワークフローを一時停止
7. **decision_manager** - 🆕 人間の応答に基づいてルーティングするカスタムエグゼキューター (上記ですでに定義済み)


In [28]:
# Agent 1: Check availability with tool (same as conditional workflow)
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: NEW - Prepare human confirmation request
confirmation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user's requested destination has no available hotel rooms. "
            "Create a polite message asking if they would like to see alternative destinations nearby. "
            "Return a JSON with: destination (the unavailable city), and question (a friendly yes/no question). "
            "Keep the question concise and friendly."
        ),
        response_format=ConfirmationQuestion,  # Use Pydantic model for agent output
    ),
    id="confirmation_agent",
)

# Agent 3: Suggest alternative (when user says yes)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 4: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

# Agent 5: NEW - Handle cancellation when user declines alternatives
class CancellationMessage(BaseModel):
    """Message when user declines alternatives."""
    status: str
    message: str

cancellation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user has declined to see alternative hotel destinations. "
            "Create a polite cancellation message. "
            "Return JSON with: status (should be 'cancelled'), and message (a friendly acknowledgment). "
            "Keep the message brief and understanding."
        ),
        response_format=CancellationMessage,
    ),
    id="cancellation_agent",
)

# NEW: RequestInfoExecutor - pauses workflow to gather human input
request_info_executor = RequestInfoExecutor(id="request_info")

# NEW: DecisionManager instance - routes based on human feedback
decision_manager = DecisionManager(id="decision_manager")

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created Workflow Components:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>confirmation_agent</strong> 🆕 - Prepares human confirmation request</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
            <li><strong>cancellation_agent</strong> 🆕 - Handles user declining alternatives</li>
            <li><strong>request_info_executor</strong> 🆕 - Pauses workflow for human input</li>
            <li><strong>decision_manager</strong> 🆕 - Routes based on human response</li>
        </ul>
    </div>
""")
)

## ステップ9: ヒューマンインザループを含むワークフローの構築

ここでは、**条件付きルーティング**を使用してヒューマンインザループの経路を含むワークフローグラフを構築します。

**ワークフロー構造:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙                    ↘
[no_availability]        [has_availability]
        ↓                        ↓
confirmation_agent          booking_agent
        ↓                        ↓
prepare_human_request      display_result
        ↓
request_info_executor (PAUSE)
        ↓
decision_manager
   ↙         ↘
[yes]        [no]
   ↓           ↓
alternative  cancellation
   ↓           ↓
display_result
```

**主要なエッジ:**
- `availability_agent → confirmation_agent` (部屋がない場合)
- `confirmation_agent → prepare_human_request` (タイプ変換)
- `prepare_human_request → request_info_executor` (人間の介入のための一時停止)
- `request_info_executor → decision_manager` (常に - RequestResponse を提供)
- `decision_manager → alternative_agent` (ユーザーが「はい」と言った場合)
- `decision_manager → cancellation_agent` (ユーザーが「いいえ」と言った場合)
- `availability_agent → booking_agent` (部屋が利用可能な場合)
- すべての経路は `display_result` で終了


In [29]:
# Build the workflow with human-in-the-loop routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    
    # NO AVAILABILITY PATH (with human-in-the-loop)
    .add_edge(availability_agent, confirmation_agent, condition=no_availability_condition)
    .add_edge(confirmation_agent, prepare_human_request)  # Transform to HumanFeedbackRequest
    .add_edge(prepare_human_request, request_info_executor)  # Send to RequestInfoExecutor
    .add_edge(request_info_executor, decision_manager)    # Always goes to decision manager
    
    # Decision manager routes based on user response
    .add_edge(decision_manager, alternative_agent, condition=user_wants_alternatives_condition)
    .add_edge(decision_manager, cancellation_agent, condition=user_declines_alternatives_condition)
    .add_edge(alternative_agent, display_result)
    .add_edge(cancellation_agent, display_result)
    
    # HAS AVAILABILITY PATH (no human input needed)
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Human-in-the-Loop Routing:</strong><br>
            • If <strong>NO availability</strong> → confirmation_agent → prepare_human_request → request_info_executor → <strong>PAUSE FOR HUMAN</strong> → decision_manager<br>
            &nbsp;&nbsp;• If user says <strong>YES</strong> → alternative_agent → display_result<br>
            &nbsp;&nbsp;• If user says <strong>NO</strong> → cancellation_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result (no human input needed)
        </p>
    </div>
""")
)

## ステップ10: テストケース1を実行 - 空き状況なしの都市 (パリ、人間による確認)

このテストは、**人間が介入する完全なサイクル**を示します:

1. パリでホテルをリクエスト
2. availability_agent が確認 → 空室なし
3. confirmation_agent が人間向けの質問を作成
4. request_info_executor が**ワークフローを一時停止**し、`RequestInfoEvent`を発行
5. **アプリケーションがイベントを検出し、コンソールでユーザーに促す**
6. ユーザーが「yes」または「no」と入力
7. アプリケーションが`send_responses_streaming()`を通じて応答を送信
8. decision_manager が応答に基づいてルーティング
9. 最終結果が表示される

**重要なパターン:**
- 初回のイテレーションには`workflow.run_stream()`を使用
- 以降のイテレーションには`workflow.send_responses_streaming(pending_responses)`を使用
- 人間の入力が必要な場合は`RequestInfoEvent`を監視
- 最終結果を取得するには`WorkflowOutputEvent`を監視


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability - Human-in-the-Loop)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → confirmation_agent → request_info_executor → <strong>PAUSE</strong> → decision_manager → (depends on user input)</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], 
    should_respond=True
)

# Human-in-the-loop execution pattern
pending_responses: dict[str, str] | None = None
completed = False
workflow_output: str | None = None

print("\n🔄 Starting human-in-the-loop workflow...")
print("=" * 60)

while not completed:
    # First iteration uses run_stream with the request
    # Subsequent iterations use send_responses_streaming with collected human responses
    if pending_responses:
        print(f"\n📤 Sending human responses: {pending_responses}")
        stream = workflow.send_responses_streaming(pending_responses)
        pending_responses = None  # Clear immediately after sending
    else:
        print(f"\n🚀 Starting workflow with request: 'I want to book a hotel in Paris'")
        stream = workflow.run_stream(request_paris)
    
    # Collect all events from this iteration
    events = [event async for event in stream]
    
    # Process events
    requests: list[tuple[str, str]] = []  # (request_id, prompt)
    
    for event in events:
        # Check for human input requests
        if isinstance(event, RequestInfoEvent) and isinstance(event.data, HumanFeedbackRequest):
            print(f"\n⏸️  WORKFLOW PAUSED - Human input requested!")
            print(f"   Request ID: {event.request_id}")
            print(f"   Destination: {event.data.destination}")
            requests.append((event.request_id, event.data.prompt))
        
        # Check for workflow outputs
        elif isinstance(event, WorkflowOutputEvent):
            workflow_output = str(event.data)
            completed = True
            print(f"\n✅ Workflow completed with output!")
    
    # If we have human requests, prompt the user
    if requests and not completed:
        responses: dict[str, str] = {}
        for req_id, prompt in requests:
            print(f"\n{'='*60}")
            print(f"💬 QUESTION FOR YOU:")
            print(f"   {prompt}")
            print(f"{'='*60}")
            
            # Get user input (in notebook, this will pause execution)
            answer = input("👉 Enter 'yes' or 'no': ").strip().lower()
            
            print(f"\n📝 You answered: {answer}")
            responses[req_id] = answer
        
        pending_responses = responses

print(f"\n{'='*60}")
print(f"🏆 FINAL WORKFLOW OUTPUT:")
print(f"{'='*60}")

# Display final result
if workflow_output:
    # Try to parse as JSON for pretty display
    try:
        result_data = json.loads(workflow_output)
        if "alternative_destination" in result_data:
            result_obj = AlternativeResult.model_validate_json(workflow_output)
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> ✅ Accepted alternatives</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_obj.alternative_destination}</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_obj.reason}</p>
                    </div>
                </div>
            """)
            )
        else:
            # User declined
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #f44336 0%, #e91e63 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(244,67,54,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> 🚫 Declined alternatives</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Result:</strong> Booking request cancelled</p>
                    </div>
                </div>
            """)
            )
    except:
        print(workflow_output)


🔄 Starting human-in-the-loop workflow...

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: 032c8fce-b9d1-400e-ba8d-afd2248e2926
   Destination: Paris

💬 QUESTION FOR YOU:
   Unfortunately, there are no rooms available in Paris. Would you like to explore nearby alternative destinations?

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: cf48dad0-ee5e-4f60-8806-341a7a292bd4
   Destination: Paris

💬 QUESTION FOR YOU:
   I'm sorry to inform you that there are no available hotel rooms in Paris. Would you like me to suggest nearby alternative destinations?

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'


## ステップ11: テストケース2を実行 - 空室ありの都市 (ストックホルム - 人間の入力不要)

このテストは、部屋が空いている場合の**直接ルート**を示します:

1. ストックホルムのホテルをリクエスト
2. availability_agent が確認 → 部屋が空いている ✅
3. booking_agent が予約を提案
4. display_result が確認を表示
5. **人間の入力は不要！**

部屋が空いている場合、ワークフローは完全に人間を介するルートをスキップします。


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability - No Human Input)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result (direct, no pause)</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], 
    should_respond=True
)

# Run the workflow (should complete without human input)
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm - No Human Input)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0 0 10px 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
                <p style='margin: 10px 0 0 0; font-size: 12px; color: #999; font-style: italic;'>Note: No human input was requested because rooms were available!</p>
            </div>
        </div>
    """)
    )

## 重要なポイントとHuman-in-the-Loopのベストプラクティス

### ✅ 学んだこと:

#### 1. **RequestInfoExecutorパターン**
Microsoft Agent FrameworkのHuman-in-the-Loopパターンは、以下の3つの主要コンポーネントを使用します:
- `RequestInfoExecutor` - ワークフローを一時停止し、イベントを発行
- `RequestInfoMessage` - 型付きリクエストペイロードの基底クラス（これをサブクラス化する！）
- `RequestResponse` - 人間の応答を元のリクエストと関連付ける

**重要な理解ポイント:**
- `RequestInfoExecutor`は自分で入力を収集しません - ワークフローを一時停止するだけです
- アプリケーションコードは`RequestInfoEvent`をリッスンして入力を収集する必要があります
- `request_id`をユーザーの回答にマッピングする辞書を使って`send_responses_streaming()`を呼び出す必要があります

#### 2. **ストリーミング実行パターン**
```python
# First iteration
stream = workflow.run_stream(initial_request)

# Subsequent iterations (after human input)
stream = workflow.send_responses_streaming(pending_responses)

# Always process events
events = [event async for event in stream]
```

#### 3. **イベント駆動型アーキテクチャ**
特定のイベントをリッスンしてワークフローを制御します:
- `RequestInfoEvent` - 人間の入力が必要（ワークフローが一時停止）
- `WorkflowOutputEvent` - 最終結果が利用可能（ワークフロー完了）
- `WorkflowStatusEvent` - 状態変更（IN_PROGRESS、IDLE_WITH_PENDING_REQUESTSなど）

#### 4. **@handlerを使ったカスタムExecutor**
`DecisionManager`は以下を示します:
- `@handler`デコレーターを使用してメソッドをワークフローステップとして公開
- 型付きメッセージを受け取る（例: `RequestResponse[HumanFeedbackRequest, str]`）
- 他のExecutorにメッセージを送信してワークフローをルーティング
- `WorkflowContext`を介してコンテキストにアクセス

#### 5. **人間の判断による条件付きルーティング**
人間の応答を評価する条件関数を作成できます:
```python
def user_wants_alternatives_condition(message: Any) -> bool:
    response_text = message.agent_run_response.text.lower()
    return response_text == "yes"
```

### 🎯 実際の応用例:

1. **承認ワークフロー**
   - 経費報告を処理する前に管理者の承認を得る
   - 自動メールを送信する前に人間によるレビューを要求
   - 高額取引を実行する前に確認を求める

2. **コンテンツモデレーション**
   - 問題のあるコンテンツを人間によるレビューにフラグ付け
   - 境界ケースについて最終決定をモデレーターに依頼
   - AIの信頼度が低い場合に人間にエスカレーション

3. **カスタマーサービス**
   - AIがルーチンの質問を自動的に処理
   - 複雑な問題を人間のエージェントにエスカレーション
   - 顧客に人間と話したいかどうか尋ねる

4. **データ処理**
   - 曖昧なデータエントリを人間に解決してもらう
   - 不明瞭な文書のAI解釈を確認
   - 複数の有効な解釈からユーザーに選択させる

5. **安全性が重要なシステム**
   - 取り返しのつかない操作の前に人間の確認を要求
   - 敏感なデータにアクセスする前に承認を得る
   - 規制業界（医療、金融）での決定を確認

6. **インタラクティブエージェント**
   - フォローアップ質問をする会話型ボットを構築
   - ユーザーを複雑なプロセスに導くウィザードを作成
   - 人間とステップバイステップで協力するエージェントを設計

### 🔄 比較: Human-in-the-Loopあり vs なし

| 機能 | 条件付きワークフロー | Human-in-the-Loopワークフロー |
|------|---------------------|---------------------------|
| **実行** | 単一の`workflow.run()` | `run_stream()` + `send_responses_streaming()`のループ |
| **ユーザー入力** | なし（完全自動化） | `input()`やUIを介したインタラクティブなプロンプト |
| **コンポーネント** | Agents + Executors | + RequestInfoExecutor + DecisionManager |
| **イベント** | AgentExecutorResponseのみ | RequestInfoEvent, WorkflowOutputEventなど |
| **一時停止** | 一時停止なし | RequestInfoExecutorでワークフローが一時停止 |
| **人間の制御** | 人間の制御なし | 人間が重要な決定を行う |
| **ユースケース** | 自動化 | コラボレーションと監視 |

### 🚀 高度なパターン:

#### 複数の人間の判断ポイント
同じワークフロー内に複数の`RequestInfoExecutor`ノードを配置できます:
```python
.add_edge(agent1, request_info_1)  # First human decision
.add_edge(decision_manager_1, agent2)
.add_edge(agent2, request_info_2)  # Second human decision
.add_edge(decision_manager_2, final_agent)
```

#### タイムアウト処理
人間の応答にタイムアウトを実装:
```python
import asyncio

try:
    answer = await asyncio.wait_for(
        asyncio.to_thread(input, "Enter yes/no: "),
        timeout=60.0
    )
except asyncio.TimeoutError:
    answer = "no"  # Default to safe option
```

#### リッチUI統合
`input()`の代わりに、Web UI、Slack、Teamsなどと統合:
```python
if isinstance(event, RequestInfoEvent):
    # Send notification to user's preferred channel
    await slack_client.send_message(
        user_id=current_user,
        text=event.data.prompt,
        request_id=event.request_id
    )
```

#### 条件付きHuman-in-the-Loop
特定の状況でのみ人間の入力を要求:
```python
def needs_human_approval_condition(message: Any) -> bool:
    # Only route to human if confidence is low or value is high
    if result.confidence < 0.7 or result.value > 10000:
        return True
    return False
```

### ⚠️ ベストプラクティス:

1. **RequestInfoMessageを必ずサブクラス化**
   - 型の安全性と検証を提供
   - UIレンダリングのためのリッチなコンテキストを可能に
   - 各リクエストタイプの意図を明確化

2. **わかりやすいプロンプトを使用**
   - 質問内容に関するコンテキストを含める
   - 各選択肢の結果を説明
   - 質問を簡潔で明確に保つ

3. **予期しない入力を処理**
   - ユーザーの応答を検証
   - 無効な入力に対するデフォルトを提供
   - 明確なエラーメッセージを表示

4. **リクエストIDを追跡**
   - request_idと応答の関連付けを使用
   - 状態を手動で管理しようとしない

5. **非ブロッキング設計**
   - 入力を待つ間スレッドをブロックしない
   - 非同期パターンを全体で使用
   - 同時ワークフローインスタンスをサポート

### 📚 関連概念:

- **Agent Middleware** - エージェント呼び出しをインターセプト（前のノートブック）
- **ワークフロー状態管理** - 実行間でワークフロー状態を保持
- **マルチエージェントコラボレーション** - Human-in-the-Loopをエージェントチームと組み合わせる
- **イベント駆動型アーキテクチャ** - イベントを使用してリアクティブシステムを構築

---

### 🎓 おめでとうございます！

Microsoft Agent Frameworkを使用したHuman-in-the-Loopワークフローを習得しました！以下のことができるようになりました:
- ✅ ワークフローを一時停止して人間の入力を収集
- ✅ RequestInfoExecutorとRequestInfoMessageを使用
- ✅ イベントを使用したストリーミング実行を処理
- ✅ @handlerを使ったカスタムExecutorを作成
- ✅ 人間の判断に基づいてワークフローをルーティング
- ✅ 人間と協力するインタラクティブなAIエージェントを構築

**これは信頼性が高く、制御可能なAIシステムを構築するための重要なパターンです！** 🚀



---

**免責事項**:  
この文書はAI翻訳サービス[Co-op Translator](https://github.com/Azure/co-op-translator)を使用して翻訳されています。正確性を追求しておりますが、自動翻訳には誤りや不正確な部分が含まれる可能性があります。元の言語で記載された文書を正式な情報源としてご参照ください。重要な情報については、専門の人間による翻訳を推奨します。この翻訳の使用に起因する誤解や誤解釈について、当社は責任を負いません。
